## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [1]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen




In [2]:
PYSAL_RELEASE = '2021-01-17'
release_date = datetime.strptime(PYSAL_RELEASE+" 0:0:0", "%Y-%m-%d %H:%M:%S")

In [3]:
release_date

datetime.datetime(2021, 1, 17, 0, 0)

In [4]:
CWD = os.path.abspath(os.path.curdir)

In [5]:
CWD

'/home/serge/projects/pysal/tools'

In [6]:
start_date = '2020-07-29'
since_date = '--since="{start}"'.format(start=start_date)
since_date
since = datetime.strptime(start_date+" 0:0:0", "%Y-%m-%d %H:%M:%S")
since

datetime.datetime(2020, 7, 29, 0, 0)

In [7]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [8]:
packages

{'libpysal': '4.3.5',
 'access': '1.1.1',
 'esda': '2.3.5',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '1.5.0',
 'spaghetti': '1.5.6',
 'mgwr': '2.1.2',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.2',
 'spvcm': '0.3.0',
 'tobler': '0.5.4',
 'mapclassify': '2.4.2',
 'splot': '.1.1.3',
 'spopt': '0.0.5'}

In [9]:
packages['pysal'] = '2.4.0.rc1'

In [10]:
import pickle

In [11]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [12]:
type(issues_closed)

dict

In [13]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [14]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [15]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [16]:
pypi_releases = get_pypi_info()

In [17]:
from datetime import datetime

In [18]:
pysal_date = datetime.strptime('2021-01-17T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [19]:
pysal_rel = {'version': 'v2.4.0rc1',
            'release_date': pysal_date}
github_releases['pysal'] = pysal_rel

In [20]:
github_releases

{'libpysal': {'version': 'v4.3.5',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.3.5',
  'release_date': datetime.datetime(2021, 1, 17, 4, 6, 18)},
 'access': {'version': 'v1.1.1',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.1',
  'release_date': datetime.datetime(2020, 7, 12, 17, 23, 50)},
 'esda': {'version': 'v2.3.5',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.3.5',
  'release_date': datetime.datetime(2020, 12, 30, 3, 18, 36)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},


In [21]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

libpysal 2021-01-17 04:06:18 100 43 0 0
access 2020-07-12 17:23:50 14 0 7 0
esda 2020-12-30 03:18:36 45 13 28 8
giddy 2020-06-10 04:59:45 34 0 0 0
inequality 2018-10-31 22:28:18 0 0 0 0
pointpats 2020-07-27 22:17:33 11 0 8 0
segregation 2020-12-30 03:31:01 20 13 0 0
spaghetti 2021-01-04 20:50:06 100 48 0 0
mgwr 2020-09-08 21:20:34 6 3 4 2
spglm 2020-09-08 20:34:08 4 3 3 3
spint 2020-09-09 02:28:50 5 2 3 2
spreg 2021-01-15 19:15:15 33 16 17 9
spvcm 2020-02-02 19:42:39 0 0 0 0
tobler 2020-12-22 18:14:45 70 28 46 17
mapclassify 2020-12-22 23:35:40 52 20 30 13
splot 2020-03-23 11:53:30 22 0 13 0
spopt 2021-01-17 18:44:46 100 74 62 41
pysal 2021-01-17 12:00:00 79 14 0 0


In [22]:
issue_details = final_issues
pull_details = final_pulls

In [23]:
packages

{'libpysal': '4.3.5',
 'access': '1.1.1',
 'esda': '2.3.5',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '1.5.0',
 'spaghetti': '1.5.6',
 'mgwr': '2.1.2',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.2',
 'spvcm': '0.3.0',
 'tobler': '0.5.4',
 'mapclassify': '2.4.2',
 'splot': '.1.1.3',
 'spopt': '0.0.5',
 'pysal': '2.4.0.rc1'}

In [24]:
github_releases['pysal']['release_date'] = release_date

In [25]:
released

datetime.datetime(2021, 1, 17, 12, 0)

In [26]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    #print(tag_date)
    #tag_date = tag_dates[subpackage]
    os.chdir(CWD)
    os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
    cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
    ncommits = len(check_output(cmd_until).splitlines())
    ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

libpysal 145 145 2021-01-17
access 53 0 2020-07-12
esda 21 21 2020-12-30
giddy 8 0 2020-06-10
inequality 0 0 2018-10-31
pointpats 0 0 2020-07-27
segregation 38 35 2020-12-30
spaghetti 150 146 2021-01-04
mgwr 5 5 2020-09-08
spglm 6 6 2020-09-08
spint 14 11 2020-09-09
spreg 95 95 2021-01-15
spvcm 0 0 2020-02-02
tobler 143 61 2020-12-22
mapclassify 61 61 2020-12-22
splot 13 0 2020-03-23
spopt 191 191 2021-01-17
pysal 37 30 2021-01-17


In [27]:
CWD

'/home/serge/projects/pysal/tools'

In [28]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    tag_date = tag_dates[subpackage]
    os.chdir(CWD)
    os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
    cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
    ncommits = len(check_output(cmd_until).splitlines())
    ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

libpysal 145 145 2021-01-17
access 53 0 2020-07-12
esda 21 21 2020-12-30
giddy 8 0 2020-06-10
inequality 0 0 2018-10-31
pointpats 0 0 2020-07-27
segregation 38 35 2020-12-30
spaghetti 150 146 2021-01-04
mgwr 5 5 2020-09-08
spglm 6 6 2020-09-08
spint 14 11 2020-09-09
spreg 95 95 2021-01-15
spvcm 0 0 2020-02-02
tobler 143 61 2020-12-22
mapclassify 61 61 2020-12-22
splot 13 0 2020-03-23
spopt 191 191 2021-01-17
pysal 37 30 2021-01-17


In [29]:
cmd_until

['git', 'log', '--oneline', '--since="2020-07-29"', '--until="2021-01-17"']

In [30]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [31]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [32]:
author_cmd.append('blank')

In [33]:
author_cmd

['git', 'log', '--format=* %aN', '--since="2020-07-29"', 'blank']

In [34]:
from collections import Counter

In [35]:
tag_dates

{'libpysal': '2021-01-17',
 'access': '2020-07-12',
 'esda': '2020-12-30',
 'giddy': '2020-06-10',
 'inequality': '2018-10-31',
 'pointpats': '2020-07-27',
 'segregation': '2020-12-30',
 'spaghetti': '2021-01-04',
 'mgwr': '2020-09-08',
 'spglm': '2020-09-08',
 'spint': '2020-09-09',
 'spreg': '2021-01-15',
 'spvcm': '2020-02-02',
 'tobler': '2020-12-22',
 'mapclassify': '2020-12-22',
 'splot': '2020-03-23',
 'spopt': '2021-01-17',
 'pysal': '2021-01-17'}

In [36]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    os.chdir(CWD)
    os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
    ncommits = len(check_output(cmd).splitlines())
    tag_date = tag_dates[subpackage]
    tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
    author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
    #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
    print(author_cmd)


    all_authors = check_output(author_cmd).splitlines()
    counter = Counter([regularize_identity(author) for author in all_authors])
    global_counter += counter
    counters.update({subpackage: counter})
    unique_authors = sorted(set(all_authors))
    authors[subpackage] =  unique_authors
    authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2021-01-18"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-07-13"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-12-31"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-06-11"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2018-11-01"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-07-28"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-12-31"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2021-01-05"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-09-09"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-09-09"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2020-09-10"']
['git', 'log', '--format=* %aN', '--since="2020-07-29"', '--until="2021-01-16"']
['git', 'log', '--format=* %

In [37]:
counter

Counter({'Serge Rey': 26, 'James Gaboardi': 5, 'Andrew Annex': 3, 'Serge': 3})

In [38]:
authors_global

{b'* Andrew Annex',
 b'* Dani Arribas-Bel',
 b'* JCS Laptop',
 b'* James Gaboardi',
 b'* Martin Fleischmann',
 b'* MgeeeeK',
 b'* Omar',
 b'* Pedro Amaral',
 b'* Serge',
 b'* Serge Rey',
 b'* Sergio Rey',
 b'* Stefanie Lumnitz',
 b'* Taylor Oshan',
 b'* Wei Kang',
 b'* Xin (Selena) Feng',
 b'* Xin Feng',
 b'* Ziqi Li',
 b'* dependabot[bot]',
 b'* eli knaap',
 b'* knaaptime',
 b'* ljwolf',
 b'* pabloestradac',
 b'* rwei5',
 b'* weikang9009'}

In [39]:
activity

{'libpysal': 145,
 'access': 53,
 'esda': 21,
 'giddy': 8,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 38,
 'spaghetti': 150,
 'mgwr': 5,
 'spglm': 6,
 'spint': 14,
 'spreg': 95,
 'spvcm': 0,
 'tobler': 143,
 'mapclassify': 61,
 'splot': 13,
 'spopt': 191,
 'pysal': 37}

In [40]:
counters

{'libpysal': Counter({'Serge Rey': 14,
          'James Gaboardi': 105,
          'Serge': 4,
          'Dependabot[Bot]': 12,
          'Martin Fleischmann': 6,
          'Stefanie Lumnitz': 1,
          'Mgeeeek': 3}),
 'access': Counter(),
 'esda': Counter({'James Gaboardi': 4,
          'Dependabot[Bot]': 3,
          'Serge Rey': 8,
          'Serge': 4,
          'Martin Fleischmann': 1,
          'Levi John Wolf': 1}),
 'giddy': Counter(),
 'inequality': Counter(),
 'pointpats': Counter(),
 'segregation': Counter({'Eli Knaap': 27,
          'James Gaboardi': 6,
          'Serge': 1,
          'Serge Rey': 1,
          'Knaaptime': 1}),
 'spaghetti': Counter({'James Gaboardi': 132,
          'Serge Rey': 1,
          'Dependabot[Bot]': 11,
          'Omar': 2}),
 'mgwr': Counter({'Taylor Oshan': 4, 'Wei Kang': 1}),
 'spglm': Counter({'Taylor Oshan': 4, 'Ziqi Li': 1, 'Wei Kang': 1}),
 'spint': Counter({'Taylor Oshan': 11}),
 'spreg': Counter({'James Gaboardi': 60,
          'Pedro

In [41]:
counters

{'libpysal': Counter({'Serge Rey': 14,
          'James Gaboardi': 105,
          'Serge': 4,
          'Dependabot[Bot]': 12,
          'Martin Fleischmann': 6,
          'Stefanie Lumnitz': 1,
          'Mgeeeek': 3}),
 'access': Counter(),
 'esda': Counter({'James Gaboardi': 4,
          'Dependabot[Bot]': 3,
          'Serge Rey': 8,
          'Serge': 4,
          'Martin Fleischmann': 1,
          'Levi John Wolf': 1}),
 'giddy': Counter(),
 'inequality': Counter(),
 'pointpats': Counter(),
 'segregation': Counter({'Eli Knaap': 27,
          'James Gaboardi': 6,
          'Serge': 1,
          'Serge Rey': 1,
          'Knaaptime': 1}),
 'spaghetti': Counter({'James Gaboardi': 132,
          'Serge Rey': 1,
          'Dependabot[Bot]': 11,
          'Omar': 2}),
 'mgwr': Counter({'Taylor Oshan': 4, 'Wei Kang': 1}),
 'spglm': Counter({'Taylor Oshan': 4, 'Ziqi Li': 1, 'Wei Kang': 1}),
 'spint': Counter({'Taylor Oshan': 11}),
 'spreg': Counter({'James Gaboardi': 60,
          'Pedro

In [42]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [43]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [44]:
sub

'pysal'

In [45]:
os.chdir(CWD)

import pandas

In [46]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [47]:
df.head()

,package,commits,total issues,pulls
0,libpysal,145,43,0
1,access,53,0,0
2,esda,21,13,8
3,giddy,8,0,0
4,inequality,0,0,0


In [48]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [49]:
df.sum()

package         libpysalaccessesdagiddyinequalitypointpatssegr...
commits                                                       980
total issues                                                  277
pulls                                                          95
dtype: object

In [50]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [51]:
contributor_table.to_html('./contributor_table.html')

In [52]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

,commits
Andrew Annex,3
Dani Arribas-Bel,2
Dependabot[Bot],26
Eli Knaap,89
James Gaboardi,400
Jcs Laptop,1
Knaaptime,5
Levi John Wolf,1
Martin Fleischmann,18
Mgeeeek,3


In [53]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [54]:
totals

Serge Rey             109
James Gaboardi        400
Serge                  21
Dependabot[Bot]        26
Martin Fleischmann     18
Stefanie Lumnitz       26
Mgeeeek                 3
Levi John Wolf          1
Eli Knaap              89
Knaaptime               5
Omar                    2
Taylor Oshan           19
Wei Kang                3
Ziqi Li                 1
Pedro Amaral           24
Pabloestradac          10
Dani Arribas-Bel        2
Jcs Laptop              1
Xin (Selena) Feng      21
Xin Feng               38
Rwei5                   2
Andrew Annex            3
dtype: int64

In [55]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [56]:
n_commits

980

In [57]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues,'  
    ' together with {n_pulls} pull requests since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        n_pulls=n_pulls, since_date = start_date))

In [58]:
line

'Overall, there were 980 commits that closed 277 issues, together with 95 pull requests since our last release on 2020-07-29.\n'

## append html files to end of changes.md with tags for toc

In [59]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    
